In [ ]:

# Import Library
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import pickle

# Load Dataset
# Pastikan file CSV hasil scraping ulasan aplikasi Maxim sudah tersedia
data = pd.read_csv('taxsee_reviews.csv')

# Data Cleaning dan Preprocessing
def clean_text(text):
    text = re.sub(r'http\S+|www.\S+', '', text)  # Hapus URL
    text = re.sub(r'[^A-Za-z\s]', '', text)      # Hapus karakter non-alfabet
    text = text.lower().strip()                 # Konversi ke huruf kecil dan hapus spasi berlebih
    return text

data['content'] = data['content'].astype(str).apply(clean_text)  # Kolom ulasan adalah 'content'

# Menambahkan Label Sentimen (Contoh: Positif jika skor > 3, Negatif jika skor <= 3)
data['sentiment'] = data['score'].apply(lambda x: 'positive' if x > 3 else 'negative')

# Membagi Data untuk Pelatihan dan Pengujian
X = data['content']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Model: Naive Bayes
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Prediksi dan Evaluasi
y_pred = model.predict(X_test_tfidf)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

# Simpan Model dan TF-IDF untuk Deployment
pickle.dump(model, open('maxim_sentiment_model.pkl', 'wb'))
pickle.dump(tfidf, open('maxim_tfidf_vectorizer.pkl', 'wb'))

print("Model dan vectorizer berhasil disimpan.")
